<a href="https://colab.research.google.com/github/kocavs/DL_MiniProject/blob/JD5226Branch/DL_Mini_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet For CIFAR0-10

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/DL_Mini_Proj
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/DL_Mini_Proj
checkpoint  data  DL_Mini_Proj.ipynb  __pycache__  renet.py


In [2]:
import torch
import torch.nn as nn
import numpy as np
#import renet

import torchvision
import torchvision.transforms as transforms

import os
from torchsummary import summary
import torch.utils.data as data

### Model Construction

In [3]:
class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionModule, self).__init__()

        # Branch 1: 1x1 convolution
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

        # Branch 2: 1x1 convolution followed by 3x3 convolution
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        branch1_out = self.branch1(x)
        branch2_out = self.branch2(x)

        return torch.cat([branch1_out, branch2_out], 1)

class InceptionResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(InceptionResNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 64, 1)
        self.layer2 = self._make_layer(128, 128, 2, stride=2)
        self.layer3 = self._make_layer(256, 256, 2, stride=2)
        self.layer4 = self._make_layer(512, 512, 1, stride=2)
        

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride=1):
        layers = []
        for _ in range(num_blocks):
            layers.append(InceptionModule(in_channels, out_channels))
            in_channels = out_channels * 2

        if stride == 2:
            layers.append(nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

# Initialize the Inception-ResNet model

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = InceptionResNet(num_classes=10).to(device)

In [5]:
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,472
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,160
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]           4,160
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11             [-1, 64, 8, 8]          36,928
      BatchNorm2d-12             [-1, 64, 8, 8]             128
             ReLU-13             [-1, 64, 8, 8]               0
  InceptionModule-14            [-1, 12

Construction Complete

### Data Preprocessing

In [6]:
class Cutout(object):
  """Randomly mask out one or more patches from an image.
  Args:
      n_holes (int): Number of patches to cut out of each image.
      length (int): The length (in pixels) of each square patch.
  """
  def __init__(self, n_holes, length):
      self.n_holes = n_holes
      self.length = length

  def __call__(self, img):
      """
      Args:
          img (Tensor): Tensor image of size (C, H, W).
      Returns:
          Tensor: Image with n_holes of dimension length x length cut out of it.
      """
      h = img.size(1)
      w = img.size(2)

      mask = np.ones((h, w), np.float32)

      for n in range(self.n_holes):
          y = np.random.randint(h)
          x = np.random.randint(w)

          y1 = np.clip(y - self.length // 2, 0, h)
          y2 = np.clip(y + self.length // 2, 0, h)
          x1 = np.clip(x - self.length // 2, 0, w)
          x2 = np.clip(x + self.length // 2, 0, w)

          mask[y1: y2, x1: x2] = 0.

      mask = torch.from_numpy(mask)
      mask = mask.expand_as(img)
      img = img * mask

      return img

In [7]:
def load_CIFAR10(batch_size, train_ratio):

  ROOT = './data'
  trainset = torchvision.datasets.CIFAR10(
      root = ROOT,
      train = True, 
      download = True
  )

  # Compute means and standard deviations
  means = trainset.data.mean(axis=(0,1,2)) / 255
  stds = trainset.data.std(axis=(0,1,2)) / 255
  #print(means, stds)

  # Preprocess setting
  transform_train = transforms.Compose([
      transforms.RandomCrop(32, padding=4),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean=means, std=stds),
      Cutout(n_holes=1, length=16)
  ])
  transform_test = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=means, std=stds)
  ])

  # Load the dataset
  trainset = torchvision.datasets.CIFAR10(
      root = ROOT, 
      train = True, 
      download = True, 
      transform = transform_train
  )
  testset = torchvision.datasets.CIFAR10(
      root = ROOT, 
      train = False, 
      download = True, 
      transform = transform_test
  )

  # Split trainset for validset
  n_train = int(len(trainset) * train_ratio)
  n_valid = len(trainset) - n_train
  train_dataset, valid_dataset = data.random_split(trainset, [n_train, n_valid])
  
  # Build dataloader
  train_iterator = data.DataLoader(train_dataset, batch_size)
  valid_iterator = data.DataLoader(valid_dataset, batch_size)
  test_iterator = data.DataLoader(testset, batch_size)

  return train_iterator, valid_iterator, test_iterator

In [8]:
trainloader, validloader, testloader = load_CIFAR10(batch_size=16, train_ratio=1)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [9]:
"""
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
"""

'\ntransform_train = transforms.Compose([\n    transforms.RandomCrop(32, padding=4),\n    transforms.RandomHorizontalFlip(),\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n\ntransform_test = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n'

In [10]:
"""
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train) # change transform in future
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test) # change transform in future
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')
"""

"\ntrainset = torchvision.datasets.CIFAR10(\n    root='./data', train=True, download=True, transform=transform_train) # change transform in future\ntrainloader = torch.utils.data.DataLoader(\n    trainset, batch_size=128, shuffle=True, num_workers=2)\n\ntestset = torchvision.datasets.CIFAR10(\n    root='./data', train=False, download=True, transform=transform_test) # change transform in future\ntestloader = torch.utils.data.DataLoader(\n    testset, batch_size=100, shuffle=False, num_workers=2)\n\nclasses = ('plane', 'car', 'bird', 'cat', 'deer',\n           'dog', 'frog', 'horse', 'ship', 'truck')\n"

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [12]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print("Train Set Loss:",train_loss)



In [13]:
def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc
    print('Test Set Accuracy:',acc)
    return acc


In [14]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

for epoch in range(start_epoch, start_epoch+20):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
Train Set Loss: 6103.683391094208
Saving..
Test Set Accuracy: 39.93

Epoch: 1
Train Set Loss: 5039.246768653393
Saving..
Test Set Accuracy: 50.54

Epoch: 2
Train Set Loss: 4583.945307016373
Saving..
Test Set Accuracy: 57.0

Epoch: 3
Train Set Loss: 4137.669887006283
Saving..
Test Set Accuracy: 60.73

Epoch: 4
Train Set Loss: 3810.20165219903
Saving..
Test Set Accuracy: 64.71

Epoch: 5
Train Set Loss: 3542.352277368307
Saving..
Test Set Accuracy: 66.9

Epoch: 6
Train Set Loss: 3370.3911491036415
Saving..
Test Set Accuracy: 68.71

Epoch: 7
Train Set Loss: 3204.077872633934
Saving..
Test Set Accuracy: 70.33

Epoch: 8
Train Set Loss: 3067.4765536636114
Saving..
Test Set Accuracy: 73.74

Epoch: 9
Train Set Loss: 2963.851778358221
Saving..
Test Set Accuracy: 73.81

Epoch: 10
Train Set Loss: 2876.468645066023
Saving..
Test Set Accuracy: 73.96

Epoch: 11
Train Set Loss: 2764.154967099428
Saving..
Test Set Accuracy: 74.34

Epoch: 12
Train Set Loss: 2695.6585119366646
Saving..
Test Set

In [15]:
for g in optimizer.param_groups:
    print(optimizer.param_groups[0]['lr'])
    g['lr'] = 0.001

0.009755282581475762


In [17]:
start_epoch = 20  # start from epoch 0 or last checkpoint epoch

for epoch in range(start_epoch, start_epoch+20):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 20
Train Set Loss: 1608.802843336016
Test Set Accuracy: 84.92

Epoch: 21
Train Set Loss: 1600.1520833522081
Test Set Accuracy: 84.78

Epoch: 22
Train Set Loss: 1603.364780113101
Saving..
Test Set Accuracy: 85.25

Epoch: 23
Train Set Loss: 1588.7829240709543
Test Set Accuracy: 84.84

Epoch: 24
Train Set Loss: 1568.23423044011
Test Set Accuracy: 84.67

Epoch: 25
Train Set Loss: 1573.033382985741
Test Set Accuracy: 85.03

Epoch: 26
Train Set Loss: 1551.8274260181934
Test Set Accuracy: 85.06

Epoch: 27
Train Set Loss: 1550.0797832719982
Test Set Accuracy: 84.81

Epoch: 28
Train Set Loss: 1532.4724542461336
Test Set Accuracy: 85.0

Epoch: 29
Train Set Loss: 1534.7018443811685
Test Set Accuracy: 85.04

Epoch: 30
Train Set Loss: 1527.4907198138535
Saving..
Test Set Accuracy: 85.49

Epoch: 31
Train Set Loss: 1529.2671598959714
Test Set Accuracy: 85.42

Epoch: 32
Train Set Loss: 1498.2877764441073
Test Set Accuracy: 85.29

Epoch: 33
Train Set Loss: 1524.1687925569713
Test Set Accuracy: 

In [30]:
optimizer.param_groups[0]['lr'] # adam lr after 20 epoch -> 0.009755

0.009755282581475762

In [23]:
print('Accuracy: ',test(0))

Saving..
Accuracy: 10.0
Accuracy:  10.0
